In [1]:
def get_par_estimates(path_matlab_result, last_SSF_str, M_names_opt, states, par_transformation,bool_identifiablePars_only,bool_weights,bool_test_par,bool_CIs,bool_boundaries,bool_fit_repetitions_seperately):
    #N_individuals = len(SSF_str)
    if bool_fit_repetitions_seperately == True:
        add_str = '_rep_sep'
    else:
        add_str = ''
    n_end=1
    for lssf_id in range(0,n_end):
        if np.size(last_SSF_str)==1:
            lssf_id=0
            lSSF_str = last_SSF_str
        else:
            n_end = len(last_SSF_str)
            lSSF_str = last_SSF_str[lssf_id]
        os.chdir(path_matlab_result + lSSF_str)
        P = sio.loadmat('ws_parameters'+add_str+'.mat')
        transformation_str = P['transformation_str']
        t_id_ratio=np.where(transformation_str[0,:]=='ratio')[0][0]

        p_opt = P['PAR_OPT_T']

        rn_opt = P['rate_names_opt']

        if bool_test_par:
            p_test = P['PAR_TEST_T']
            rate_names_test = P['rate_names_test']
        if par_transformation is 'ratio':
            if len(np.shape(p_opt))==2:
                p_opt[t_id_ratio,:] = p_opt[t_id_ratio,:]/24
                if bool_test_par:
                    p_test[t_id_ratio,:] = p_test[t_id_ratio,:]/24
            elif len(np.shape(p_opt))==3:
                p_opt[t_id_ratio,:,:] = p_opt[t_id_ratio,:,:]/24
                if bool_test_par:
                    p_test[t_id_ratio,:,:] = p_test[t_id_ratio,:,:]/24

        if bool_CIs or bool_boundaries or bool_identifiablePars_only or (bool_weights and par_transformation=='lin'):
            CI_l = P['CI_lower']
            CI_u = P['CI_upper']
            p_min = P['par_min']
            p_max = P['par_max']
            if par_transformation is 'ratio':
                if len(np.shape(p_opt))==2:
                    CI_l[t_id_ratio,:] = CI_l[t_id_ratio,:]/24
                    CI_u[t_id_ratio,:] = CI_u[t_id_ratio,:]/24
                    p_min[t_id_ratio,:] = p_min[t_id_ratio,:]/24
                    p_max[t_id_ratio,:] = p_max[t_id_ratio,:]/24
                elif len(np.shape(p_opt))==3:
                    CI_l[t_id_ratio,:,:] = CI_l[t_id_ratio,:,:]/24
                    CI_u[t_id_ratio,:,:] = CI_u[t_id_ratio,:,:]/24
                    p_min[t_id_ratio,:,:] = p_min[t_id_ratio,:,:]/24
                    p_max[t_id_ratio,:,:] = p_max[t_id_ratio,:,:]/24

        I_str = P['individuals_str']
        if bool_test_par:
            if last_SSF_str[lssf_id].endswith('weak_noise'):
                Indis = 'weak'
            elif last_SSF_str[lssf_id].endswith('middle_noise'):
                Indis = 'middle'
            elif last_SSF_str[lssf_id].endswith('strong_noise'):
                Indis = 'strong'
            elif last_SSF_str[lssf_id].endswith('realistic_noise'):
                Indis = 'realistic'
            #Indis=last_SSF_str[lssf_id]
            if len(np.shape(p_opt))==2:
                N_individuals=1
            elif len(np.shape(p_opt))==3:
                N_individuals=np.shape(p_opt)[2]
        else:
            if lssf_id==0:
                Indis = list()
                for row in I_str.flatten():
                    Indis.append('H'+row.astype(str)[0])
                N_individuals=len(Indis)
            else:
                Indis = list()
                for row in I_str.flatten():
                    Indis.append('MDS'+row.astype(str)[0])
                N_individuals=len(Indis)

        T_ID=np.where(transformation_str[0,:]==par_transformation)[0][0]
        
        rn_opt_new = (np.concatenate(np.concatenate(rn_opt)))

        rate_names_opt = list()
        
        for r_id in range(0,np.shape(rn_opt_new)[0]):
            if not rn_opt_new[r_id].startswith('x0_'):
                rate_names_opt.append(rn_opt_new[r_id])

        if len(np.shape(p_opt))==2:
            [a,b]=np.shape(p_opt)
        elif len(np.shape(p_opt))==3:
            [a,b,c]=np.shape(p_opt)
        
        DF_temp = pd.DataFrame(data=['model_'+str(M_names_opt[0])]*(N_individuals),columns=['Model'])
        DF_temp['Sample_ID_long']=Indis
        DF_temp['Sample_ID']=Indis
        #DF_temp['Sample_ID']=[name[:-2] for name in Indis] 
    for p_id in range(0,b):
        colName=str(rate_names_opt[p_id].item(0).replace('{','').replace('}',''))
        if len(np.shape(p_opt))==2:
            DF_temp[colName] = p_opt[T_ID,p_id]
            if bool_identifiablePars_only and ~(CI_l[T_ID,p_id] > p_min[T_ID,p_id] and CI_u[T_ID,p_id] < p_max[T_ID,p_id]):
                DF_temp[colName] = 'NaN'
            if bool_weights and par_transformation=='lin': 
                W = np.transpose((p_max[T_ID,p_id]-p_min[T_ID,p_id])/(CI_u[T_ID,p_id]-CI_l[T_ID,p_id]))
            if bool_CIs:
                DF_temp[colName+'_CI_l'] = CI_l[T_ID,p_id]
                DF_temp[colName+'_CI_u'] = CI_u[T_ID,p_id]
            if bool_boundaries:
                DF_temp[colName+'_min'] = p_min[T_ID,p_id]
                DF_temp[colName+'_max'] = p_max[T_ID,p_id]
            if bool_test_par:
                DF_temp[colName+'_t'] = p_test[T_ID,p_id]
        elif len(np.shape(p_opt))==3:
            DF_temp[colName] = np.transpose(p_opt[T_ID,p_id,:])
            if bool_identifiablePars_only:
                idx = [i for i in range(0,c) if ~(CI_l[T_ID,p_id,i] > p_min[T_ID,p_id,i] and CI_u[T_ID,p_id,i] < p_max[T_ID,p_id,i])]
                DF_temp[colName][idx] = 'NaN'
            if bool_weights and par_transformation=='lin': 
                W = np.transpose((p_max[T_ID,p_id,:]-p_min[T_ID,p_id,:])/(CI_u[T_ID,p_id,:]-CI_l[T_ID,p_id,:]))
            if bool_CIs:
                DF_temp[colName+'_CI_l'] = np.transpose(CI_l[T_ID,p_id,:])    
                DF_temp[colName+'_CI_u'] = np.transpose(CI_u[T_ID,p_id,:]) 
            if bool_boundaries:
                DF_temp[colName+'_min'] = np.transpose(p_min[T_ID,p_id,:])
                DF_temp[colName+'_max'] = np.transpose(p_max[T_ID,p_id,:])
            if bool_test_par:
                DF_temp[colName+'_t'] = np.transpose(p_test[T_ID,p_id,:])
        if bool_weights and par_transformation=='lin': 
            colName_W=str(rate_names_opt[p_id].item(0).replace('{','').replace('}',''))+'_w'
            DF_temp[colName_W] = W/np.sum(W)
    if lssf_id>0:
        DF_PAR = pd.concat([DF_PAR,DF_temp], axis=0, ignore_index=True)
    else:
        DF_PAR = DF_temp 
    for rate in DF_PAR.columns:
        DF_PAR = DF_PAR.replace('NaN',np.nan, regex=True)
        if rate != 'Model' and rate != 'Sample_ID' and rate != 'Sample_ID_long':
            DF_PAR[rate]=pd.to_numeric(DF_PAR[rate])
    return DF_PAR, rate_names_opt

In [5]:
def plotModelFitToData (dir_str_matlab,F_str,SF_str,SSF_str,CT,cols_CT,bool_test,opt_save):
    currentpath = os.getcwd()
    for f_id in range(0,len(F_str)):
        for sf_id in range(0,len(SF_str)):
            for ssf_id in range(0,len(SSF_str)):
                path_matlab_result = dir_str_matlab + F_str[f_id] + SF_str[sf_id] + SSF_str[ssf_id] + '/'
                os.chdir(path_matlab_result)
                #get all mat files of interest
                files_sum = [s for s in os.listdir(path_matlab_result) if "ws_modelFitPlot_sum" in s]
                files_i = [s for s in os.listdir(path_matlab_result) if "ws_modelFitPlot" in s]
                files = [s for s in files_i if s not in files_sum]
                for mat_id in range(0,len(files_sum)):
                    mat_sum = sio.loadmat(files_sum[mat_id])  
                    mat = sio.loadmat(files[mat_id])  
                    ndivs=int(mat['ndivs'][:].flatten())
                    [a,b1]=np.shape(mat['S_opt_T'])
                    [a,b2]=np.shape(mat_sum['S_opt_T'])
                    b=b1+b2
                    [r,c]=np.shape(mat['states_str'])
                    if mat['states_str'][0][c-1][0]=='D':
                        stopp_threshold = b-2
                    else:
                        stopp_threshold = b-1
                    n_cols = ndivs+1
                    n_rows = np.max(np.shape(mat['states_str']))
                    f, axes = plt.subplots(n_rows, n_cols, sharey='row', figsize=(25, 30))
                    f.patch.set_facecolor('white')
                    f.subplots_adjust(hspace = .45, wspace = 0.35)
                    i1=0
                    i2=0
                    i3=0
                    tM1 = pd.Series(mat['t'][:].flatten())/24
                    tD1 = pd.Series(mat['tD'][:].flatten())/24
                    tM2 = pd.Series(mat_sum['t'][:].flatten())/24
                    tD2 = pd.Series(mat_sum['tD'][:].flatten())/24
                    for c_id in range(0,n_rows):
                        for d_id in range(0,n_cols):
                            i=i1+i2+i3
                            if i<=stopp_threshold:
                                if d_id<(n_cols-1):
                                    YM1 = pd.Series(mat['S_opt_T'][:,i1])
                                    if c_id==0 & d_id==1:
                                        print(YM1)
                                    if bool_test:
                                        YM1_test = pd.Series(mat['S_test_T'][:,i1])
                                    s_down1 = pd.Series(mat['e_down'][:,i1])
                                    s_up1 = pd.Series(mat['e_up'][:,i1])
                                    YD1 = pd.Series(mat['YD'][:,i1])
                                    axes[c_id,d_id].plot(tM1,YM1,color=cols_CT[CT.index(mat['states_str'][0][c_id][0])])
                                    axes[c_id,d_id].fill_between(tM1, s_down1, s_up1, alpha=0.2, edgecolor=cols_CT[CT.index(mat['states_str'][0][c_id][0])], facecolor=cols_CT[CT.index(mat['states_str'][0][c_id][0])], linewidth = 2)
                                    axes[c_id,d_id].plot(tD1,YD1,color=cols_CT[CT.index(mat['states_str'][0][c_id][0])],marker='o',linestyle='')
                                    if bool_test:
                                        axes[c_id,d_id].plot(tM1,YM1_test,color=cols_CT[CT.index(mat['states_str'][0][c_id][0])],ls='--')
                                    axes[0,d_id].set_title(str(d_id)+' divisions')
                                    i1=i1+1
                                else:
                                    YM2 = pd.Series(mat_sum['S_opt_T'][:,c_id])
                                    if bool_test:
                                        YM2_test = pd.Series(mat_sum['S_test_T'][:,c_id])
                                    s_down2 = pd.Series(mat_sum['e_down'][:,c_id])
                                    s_up2 = pd.Series(mat_sum['e_up'][:,c_id])
                                    YD2 = pd.Series(mat_sum['YD'][:,c_id])
                                    axes[c_id,d_id].plot(tM2,YM2,color=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])])
                                    axes[c_id,d_id].fill_between(tM2, s_down2, s_up2, alpha=0.2, edgecolor=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])], facecolor=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])], linewidth = 2)
                                    axes[c_id,d_id].plot(tD2,YD2,color=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])],marker='o',linestyle='')
                                    if bool_test:
                                        axes[c_id,d_id].plot(tM2,YM2_test,color=cols_CT[CT.index(mat_sum['states_str'][0][c_id][0])],ls='--')
                                    axes[0,d_id].set_title('sum over divisions')
                                    i2=i2+1
                                axes[c_id,d_id].set_xlabel("Days")
                                axes[c_id,d_id].set_xlim([0,8])
                                axes[c_id,d_id].tick_params(labelsize=16)
                            else:
                                f.delaxes(axes[c_id,d_id])
                                i3=i3+1
                        axes[c_id,0].set_ylabel(mat['states_str'][0][c_id][0])
                    f.text(0.04, 0.5, str(mat_sum['trans'][0]), va='center', rotation='vertical')
                    f.suptitle(SF_str[sf_id] + SSF_str[ssf_id] + '_' + str(mat['i_repe']) + '_' + str(mat['i_repl']))
                    if opt_save:
                        fig = plt.gcf()
                        fig.savefig(path_matlab_result + '/' + 'ModelFit_' + SF_str[sf_id][0:7] + str(mat['i_repe']) + '_' + str(mat['i_repl']) + '.svg', bbox_inches="tight")
                        fig.savefig(path_matlab_result + '/' + 'ModelFit_' + SF_str[sf_id][0:7] + str(mat['i_repe']) + '_' + str(mat['i_repl']) + '.pdf', bbox_inches="tight")
                    plt.show()
    os.chdir(currentpath)
    return;

In [ ]:
def plot_par_with_CI_vs_individuals(main_folder_matlab, F_str, SF_str, SSF_str, last_SSF_str, M_names_sim, M_names_opt, states, par_transformation, colors, opt_comparison, rot_deg, bool_test_par, bool_mean_idPars, bool_weightedMean_Pars, opt_sample_ID, bool_fit_repetitions_seperately,opt_save):

    if opt_sample_ID=='short':
        I_col = 'Sample_ID'
    elif opt_sample_ID=='long':
        I_col = 'Sample_ID_long'

    W=0.2
    #specify dataframe settings
    add_str=''
    bool_weights = False
    if bool_mean_idPars:
        add_str='_mean_identifiable_pars'
    if bool_weightedMean_Pars:
        bool_weights = True
        add_str='_weighted_mean_pars'

    #initialize bools for plotting
    markerSize=15
    bool_plot_legend=True

    #define thresholds
    TH_total = 2#7 # min of identifiable parameters to draw plot
    
    if bool_test_par:
        alpha_val=0.2
    else:
        alpha_val=0.5
    #define positions for text, number of columns and rows
    if (M_names_opt[0]=='A' and bool_test_par==False) or (M_names_opt[0]=='A' and bool_test_par==True and M_names_sim[0]=='A'):
        n_cols = 4
        x_d = 0.26
        x_p = 0.59
        x_a = 0.79
    else:
        n_cols = 6
        x_d = 0.34
        x_p = 0.69
        x_a = 0.84    
    n_rows = len(states)
    for f_id in range(0,len(F_str)):
        for sf_id in range(0,len(SF_str)):
            ## get dataframe with results
            #parameters
            path_MATLAB_result = main_folder_matlab + F_str[f_id] + SF_str[sf_id] 
            
            ## specify figure settings
            if bool_test_par:
                f, axes = plt.subplots(n_rows, n_cols, figsize=(25, 45))
                f.subplots_adjust(hspace = .75, wspace = 0.45)  
                
                DF,rn = get_par_estimates(path_MATLAB_result, SSF_str, M_names_opt, CT, par_transformation, False,bool_weights,bool_test_par,True,True,bool_fit_repetitions_seperately)

                rate_names = DF.columns[np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(DF.columns.str.endswith('_CI_l')==False,DF.columns.str.endswith('_CI_u')==False),DF.columns.str.endswith('_min')==False),DF.columns.str.endswith('_max')==False),DF.columns.str.endswith('_t')==False),DF.columns.str.endswith('_w')==False),DF.columns.str.startswith('_S')==False),DF.columns.str.startswith('_M')==False)] 
                offset = [0,len(DF[I_col])]
                N_indi = len(DF[I_col])
                I_list = [i for i in range(0,len(N_indi))]
                group_str = ['test parameter', '']
            else:
                f, axes = plt.subplots(n_rows, n_cols, figsize=(55, 65))
                f.subplots_adjust(hspace = .45, wspace = 0.2)

                DF,rn = get_par_estimates(path_MATLAB_result, last_SSF_str, M_names_opt, CT, par_transformation, False ,bool_weights,bool_test_par,True,True,bool_fit_repetitions_seperately)

                rate_names = DF.columns[np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(DF.columns.str.endswith('_CI_l')==False,DF.columns.str.endswith('_CI_u')==False),DF.columns.str.endswith('_min')==False),DF.columns.str.endswith('_max')==False),DF.columns.str.endswith('_t')==False),DF.columns.str.endswith('_w')==False),DF.columns.str.startswith('_S')==False),DF.columns.str.startswith('_M')==False)] 
                
                N_indi = len(DF[I_col])
                I_list = [i for i in range(0,N_indi)]

            f.patch.set_facecolor('white')
            s_id = 0 #states
            while s_id < len(CT):
                cols_del = list()
                col_counter = 0
                for r_id,rate in enumerate(rate_names):
                    a_m = fnm.fnmatch(rate,'a_'+ CT[s_id] +'*')
                    b_m = fnm.fnmatch(rate,'b_'+ CT[s_id] +'*')
                    g_m = fnm.fnmatch(rate,'g_'+ CT[s_id] +'*')
                    if (a_m or b_m or g_m):
                        if a_m:
                            col = col_counter
                            col_counter = col_counter+1
                        if b_m:
                            col = n_cols-2
                        if g_m:
                            col = n_cols-1
                        #plot weighted group average
                        if bool_weightedMean_Pars and par_transformation is 'lin' and N_indi>TH_total:
                            w_baseline=N_indi
                            ar=DF[rate]
                            w_D = DF['weight_data']
                            w_CI = DF[rate+'_w']
                            w = (w_D+w_CI)/2
                            weighted_stats = DescrStatsW(ar, weights=w, ddof=0)
                            par_mean_w = weighted_stats.mean
                            par_std_w = weighted_stats.std
                            axes[s_id,col].add_patch(patches.Rectangle((offset[g_id]-W*2, max(DF[rate+'_min'][0],par_mean_w-par_std_w*2)), width=w_baseline-1+W*4, height=min((DF[rate+'_max'][0]-DF[rate+'_min'][0]),4*par_std_w),color=colors,alpha=0.15,label='weighted mean rate +/- 2 weighted std'))          
                            axes[s_id,col].plot([offset[g_id]-W*2,offset[g_id]+w_baseline-1+W],[par_mean_w, par_mean_w],lw=2,ls='--',color=colors,label='weighted mean rate')        

                        #plot average and std of identifiable parameters:
                        if bool_mean_idPars and par_transformation=='lin':
                                w_baseline=N_indi
                                par_id = DF[rate][np.logical_and(DF[rate+'_min']<DF[rate+'_CI_l'],DF[rate+'_max']>DF[rate+'_CI_u'])==True]
                                if len(par_id)>TH_groups:
                                    par_id_mean=np.mean(par_id)
                                    par_id_std=np.std(par_id)
                                    axes[s_id,col].add_patch(patches.Rectangle((offset[g_id]-W*2, max(DF[rate+'_min'][0],par_id_mean-par_id_std*2)), width=w_baseline-1+W*2, height=min((DF[rate+'_max'][0]-DF[rate+'_min'][0]),4*par_id_std),color=colors,alpha=0.15,label='mean of idetifiable parameters +/- 2 std'))          
                                    axes[s_id,col].plot([offset[g_id]-W*2,offset[g_id]+w_baseline-1+W],[par_id_mean, par_id_mean],lw=2,ls='--',color=colors,label='mean of identifiable rates')        

                        #plot CIs:
                        if par_transformation is not 'ratio':
                            for I_ID in I_list:
                                H = DF[rate+'_CI_u']-DF[rate+'_CI_l']
                                axes[s_id,col].add_patch(patches.Rectangle((I_ID - W/2 ,DF[rate+'_CI_l'][I_ID]), width=W, height=H[I_ID],color=colors,alpha=alpha_val,label='95% confidence interval' if I_ID==0 else ''))
                        #plot opt par:
                        if not bool_test_par:
                            mean_rate = round(np.mean(DF[rate])*1000)/1000
                            axes[s_id,col].plot([I_list[0],I_list[len(I_list)-1]],[mean_rate, mean_rate], linestyle='-', color=colors, label='mean rate estimate')
                            axes[s_id,col].plot(I_list,DF[rate], linestyle='None', marker = 'o', ms=markerSize, color=colors, label='rate estimate')
                        else:
                            axes[s_id,col].plot(I_list,DF[rate], linestyle='None', marker = 'o', ms=markerSize, color=colors, label='rate estimate')
                        x_lim = axes[s_id,col].get_xlim()

                        #plot test parameter
                        if bool_test_par and any(DF.columns==rate+'_t'):
                            axes[s_id,col].plot(I_list,DF[rate+'_t'],marker='o',ms=markerSize-2,color='crimson',  linestyle='None', label='test parameter')

                        # axes limits and labels:
                        # axes[s_id,col].set_xlabel("Sample ID")
                        if par_transformation is 'ratio':
                            axes[s_id,col].set_ylabel('1/' + rate_names[r_id])
                        elif par_transformation is 'lin':
                            axes[s_id,col].set_ylabel(rate_names[r_id])
                        else:
                            axes[s_id,col].set_ylabel(par_transformation + '(' + rate_names[r_id] + ')')
                        if par_transformation!='lin':
                            axes[s_id,col].set_ylim(DF[rate+'_min'][0], DF[rate+'_max'][0])
                        if bool_test_par:
                            axes[s_id,col].set_xlim(x_lim[0]-0.1, x_lim[1]+0.1)
                            axes[s_id,col].set_xlabel('noise level')
                        else:
                            axes[s_id,col].set_xlim(-1, len(I_list))
                        #x tick labels
                        if not bool_test_par:
                            axes[s_id,col].set_xticks(I_list)
                            axes[s_id,col].set_xticklabels(DF[I_col],rotation=rot_deg)
                        else:
                            axes[s_id,col].set_xticks(I_list)
                            axes[s_id,col].set_xticklabels(DF[I_col],rotation=rot_deg)

                        if bool_plot_legend==True:
                            if not bool_test_par:
                                s_val = 2
                            else:
                                s_val = 1.2
                            axes[s_id,col].legend(bbox_to_anchor=(s_val, 1),numpoints=1)
                            bool_plot_legend=False
                        else: 
                            axes[s_id,col].legend_ = None

                    else:   
                        if col_counter not in cols_del:
                            cols_del.append(col_counter)

                if r_id==len(rate_names)-1:
                    for i_cd in range(max(cols_del),n_cols-2):
                        f.delaxes(axes[s_id,i_cd])
                    if CT[s_id]=='MLP':
                        f.delaxes(axes[s_id,n_cols-1])
                s_id = s_id+1 #states

            # figure settings   
            f.text(0.03, 0.5, 'Model states', va='center', rotation='vertical',fontsize=40)
            if par_transformation is 'ratio':
                string1 = 'Times for Model ' + M_names_opt[sf_id]
            else:
                string1 = 'Rates for Model ' + M_names_opt[sf_id]
            f.text(0.45, 0.95, string1, va='center')
            f.text(x_d, 0.93, 'differentiation', va='center')
            f.text(x_p, 0.93, 'proliferation', va='center')
            f.text(x_a, 0.93, 'apoptosis', va='center')

            #save figure    
            if opt_save:
                fig = plt.gcf()
                fig.savefig(path_MATLAB_result + '/' + 'ParameterWithCIs_'+ par_transformation + opt_comparison + add_str +'.svg', bbox_inches="tight")
                fig.savefig(path_MATLAB_result + '/' + 'ParameterWithCIs_'+ par_transformation + opt_comparison + add_str +'.pdf', bbox_inches="tight")
            plt.show()

In [ ]:
def plot_par_with_CI_vs_N_is(DF,results_str, model, individual, N_is_list, states, par_transformation, colors, opt_save):
    
    rate_names = DF.columns[np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(np.logical_and(DF.columns!='Model', DF.columns!='Sample_ID_long'),DF.columns!='N_is'),DF.columns!='Sample_ID'),DF.columns.str.endswith('CI_l')==False),DF.columns.str.endswith('CI_u')==False),DF.columns.str.endswith('_min')==False),DF.columns.str.endswith('_max')==False)]
    W=0.2
    markerSize=15
    bool_plot_legend=True
    alpha_val=0.5
    
    #define thresholds
    TH_total = 2#7 # min of identifiable parameters to draw plot
    
    
    #define positions for text, number of columns and rows
    n_cols = 6
    x_d = 0.34
    x_p = 0.69
    x_a = 0.84    
    n_rows = len(states)
    
            
    f, axes = plt.subplots(n_rows, n_cols, figsize=(55, 65))
    f.subplots_adjust(hspace = .25, wspace = 0.4)
    f.patch.set_facecolor('white')

    s_id = 0 #states
    while s_id < len(CT):
        cols_del = list()
        col_counter = 0
        for r_id,rate in enumerate(rate_names):
            a_m = fnm.fnmatch(rate,'a_'+ CT[s_id] +'*')
            b_m = fnm.fnmatch(rate,'b_'+ CT[s_id] +'*')
            g_m = fnm.fnmatch(rate,'g_'+ CT[s_id] +'*')
            #if (not all(np.isnan(DF[rate]))) and (a_m or b_m or g_m):
            if (a_m or b_m or g_m):
                if a_m:
                    col = col_counter
                    col_counter = col_counter+1
                if b_m:
                    col = n_cols-2
                if g_m:
                    col = n_cols-1

                #plot CIs:
                if par_transformation is not 'ratio':
                    if not bool_test_par:
                        for N_ID,N_is in enumerate(N_is_list):
                            H = DF[(DF['N_is']==N_is)][rate+'_CI_u'].astype(np.float64)-DF[(DF['N_is']==N_is)][rate+'_CI_l'].astype(np.float64)
                            ey = DF[(DF['N_is']==N_is)][rate+'_CI_l'].astype(np.float64)
                            axes[s_id,col].add_patch(patches.Rectangle((N_is - W/2 ,ey), width=W, height=H,color=colors[0],alpha=alpha_val,label='95% confidence interval' if N_ID==0 else ''))

                axes[s_id,col].plot(N_is_list,DF[rate], linestyle='None', marker = 'o', ms=markerSize, color=colors[0], label='rate estimate')
                x_lim = axes[s_id,col].get_xlim()

                # axes limits and labels:
                # axes[s_id,col].set_xlabel("Sample ID")
                if par_transformation is 'ratio':
                    axes[s_id,col].set_ylabel('1/' + rate_names[r_id])
                elif par_transformation is 'lin':
                    axes[s_id,col].set_ylabel(rate_names[r_id])
                else:
                    axes[s_id,col].set_ylabel(par_transformation + '(' + rate_names[r_id] + ')')
                #if par_transformation!='lin':
                #print(rate)
                #print([np.min(DF[rate+'_min']), np.max(DF[rate+'_max'])])
                axes[s_id,col].set_ylim([np.min(DF['b_HSC_min']), np.max(DF['b_HSC_max'])])
                axes[s_id,col].set_xlabel('N_is')

                if bool_plot_legend==True:
                    axes[s_id,col].legend(bbox_to_anchor=(2, 1),numpoints=1)
                    bool_plot_legend=False
                else: 
                    axes[s_id,col].legend_ = None

            else:   
                if col_counter not in cols_del:
                    cols_del.append(col_counter)

        if r_id==len(rate_names)-1:
            for i_cd in range(max(cols_del),n_cols-2):
                f.delaxes(axes[s_id,i_cd])
            if CT[s_id]=='MLP':
                f.delaxes(axes[s_id,n_cols-1])
        s_id = s_id+1 #states

    # figure settings   
    f.text(0.03, 0.5, 'Model states', va='center', rotation='vertical',fontsize=40)
    if par_transformation  is 'ratio':
        string1 = 'Times for Model ' + model[-1]
    else:
        string1 = 'Rates for Model ' + model[-1]
    f.text(0.45, 0.95, string1, va='center')
    f.text(x_d, 0.93, 'differentiation', va='center')
    f.text(x_p, 0.93, 'proliferation', va='center')
    f.text(x_a, 0.93, 'apoptosis', va='center')

    #save figure    
    if opt_save:
        fig = plt.gcf()
        fig.savefig(results_str + '/' + 'ParameterWithCIs_vs_N_is_'+ par_transformation + '_' + model + '_' + individual + '.svg', bbox_inches="tight")
        fig.savefig(results_str + '/' + 'ParameterWithCIs_vs_N_is_'+ par_transformation + '_' + model + '_' + individual +'.pdf', bbox_inches="tight")
    plt.show()

In [7]:
def getScores(dir_str_matlab,F_str,SF_str,SSF_str,noise_level_str,Score_label,bool_fit_repetitions_seperately,bool_intermedStates):
    if len(noise_level_str)>0:
        add_str1 = '_'+noise_level_str+'_noise'
    else:
        add_str1 = ''
    if bool_fit_repetitions_seperately:
        add_str2 = '_rep_sep'
    else:
        add_str2 = ''
    if bool_intermedStates==True:
        add_str3 = '_intermedStates'
    else:
        add_str3 = ''
    Scores = np.zeros((len(F_str),len(SF_str),len(SSF_str)))
    Scores_all = np.zeros((len(F_str),len(SF_str)))
    currentpath = os.getcwd()
    #if np.shape(folder_str)[0]==1:
    f_id=0
    for file_dir in F_str:
        path_matlab_result = dir_str_matlab + file_dir
        os.chdir(path_matlab_result)
        S = sio.loadmat('ws_scores'+add_str1+add_str2+add_str3+'.mat')  
        if Score_label == 'BIC':
            Scores[f_id,:,:] = S['BIC_mat']
            Scores_all[f_id,:] = S['BIC_all_mat'].flatten()
        elif Score_label == 'AIC':
            Scores[f_id,:,:] = S['AIC_mat']
            Scores_all[f_id,:] = S['AIC_all_mat'].flatten()
        elif Score_label == 'AIC_c':
            Scores[f_id,:,:] = S['AIC_corrected_mat']
            Scores_all[f_id,:] = S['AIC_corrected_all_mat'].flatten()
        elif Score_label == 'LOGL':
            Scores[f_id,:,:] = S['LOGL_mat']
            Scores_all[f_id,:] = S['LOGL_all_mat'].flatten()
        elif Score_label == 'TIME':
            Scores[f_id,:,:] = S['Time_mat']
            Scores_all[f_id,:] = np.sum(np.matrix(S['Time_mat']),axis=1).flatten()
        elif Score_label == 'TIME_CPU':
            Scores[f_id,:,:] = S['Time_mat_cpu']
            Scores_all[f_id,:] = np.sum(np.matrix(S['Time_mat_cpu']),axis=1).flatten()
        f_id=f_id+1
        Scores = np.round(Scores)
        Scores_all = np.round(Scores_all)
    return Scores, Scores_all

In [8]:
def plot_scores_vs_nis_per_model_and_individual(SSF_str_matlab,model_str,N_is,Scores,Score_label,results_str_matlab,bool_all_models,bool_fit_repetitions_seperately,opt_save):
    n_cols = len(SSF_str_matlab)#individuals
    if bool_fit_repetitions_seperately:
        n_digits = 5
    else:
        n_digits = 3
    
    #print(n_cols)
    if bool_all_models:
        n_rows = 1
        f, axes = plt.subplots(n_rows, n_cols, figsize=(40, n_rows*2+2))
        f.patch.set_facecolor('white')
        f.subplots_adjust(hspace = .45, wspace = 0.35)
        for m_id in range(0,len(model_str)):
            for i_id in range(0,n_cols):
                axes[i_id].plot(N_is,Scores[:,m_id,i_id], linestyle='--', marker='o',color=colors[m_id])
                axes[i_id].xaxis.set_major_locator(MaxNLocator(integer=True))
                if m_id==n_rows-1:
                    axes[i_id].set_xlabel("N_iS")
                    axes[i_id].set_title(SSF_str_matlab[i_id][len(SSF_str_matlab[i_id])-n_digits:len(SSF_str_matlab[i_id])])
                if i_id==0:
                    if Score_label.startswith('TIME'):
                        add_str = ' [h]'
                    else:
                        add_str = ''
                    axes[0].set_ylabel(Score_label+add_str)    
                axes[n_cols-1].legend(bbox_to_anchor=(1.3, 1),labels=model_str)
        f.text(0.5, 0.99, 'Individuals', va='center')
        if opt_save:
            fig = plt.gcf()
            fig.savefig(results_str_matlab + '/' + Score_label + '_Scores_vs_Nis_per_individual_all_models.svg', bbox_inches="tight")
            fig.savefig(results_str_matlab + '/' + Score_label + '_Scores_vs_Nis_per_individual_all_models.pdf', bbox_inches="tight")
    else:
        n_rows = len(model_str)#hierarchies
        f, axes = plt.subplots(n_rows, n_cols, figsize=(40,n_rows*3+1))
        f.patch.set_facecolor('white')
        f.subplots_adjust(hspace = .45, wspace = 0.35)
        for m_id in range(0,n_rows):
            for i_id in range(0,n_cols):
                if n_rows==1:
                    S = np.matrix(Scores)
                    axes[i_id].plot(N_is,S[:,i_id], linestyle='--', marker='o', color='lightseagreen', label=Score_label)
                    axes[i_id].set_xlabel("Number of intermediate states")
                    axes[i_id].xaxis.set_major_locator(MaxNLocator(integer=True))
                    axes[i_id].set_title(SSF_str_matlab[i_id][len(SSF_str_matlab[i_id])-n_digits:len(SSF_str_matlab[i_id])])
                    if i_id==0:
                        axes[0].set_ylabel(model_str[m_id]) 
                        axes[i_id].legend()
                else:
                    axes[m_id,i_id].plot(N_is,Scores[:,m_id,i_id], linestyle='--', marker='o', color='lightseagreen', label=Score_label)
                    axes[m_id,i_id].xaxis.set_major_locator(MaxNLocator(integer=True))
                    if m_id==n_rows-1 and n_rows>1:
                        axes[m_id,i_id].set_xlabel("Number of intermediate states")
                    if m_id==0:
                        axes[0,i_id].set_title(SSF_str_matlab[i_id][len(SSF_str_matlab[i_id])-n_digits:len(SSF_str_matlab[i_id])])
                    if i_id==0:
                        axes[m_id,0].set_ylabel(model_str[m_id]) 
                        if m_id==0:
                            axes[m_id,i_id].legend()
                            
        if Score_label.startswith('TIME'):
                add_str = ' [h]'
        else:
            add_str = ' values'
        f.text(0.08, 0.5, Score_label + add_str, va='center', rotation='vertical')
        f.text(0.5, 0.95, 'Individuals', va='center')
        if opt_save:
            fig = plt.gcf()
            fig.savefig(results_str_matlab + '/' + Score_label + '_Scores_vs_Nis_per_model_and_individual.svg', bbox_inches="tight")
            fig.savefig(results_str_matlab + '/' + Score_label + '_Scores_vs_Nis_per_model_and_individual.pdf', bbox_inches="tight")
    return;

In [9]:
def plot_scores_vs_nis_per_model(model_str,N_is,Scores_all,Score_label,path_matlab_result,bool_subplots,opt_save):
    if bool_subplots:
        n_rows = len(model_str)#hierarchies
        n_cols = 1
        f, axes = plt.subplots(n_rows, n_cols, figsize=(7, 15))
        f.subplots_adjust(hspace = .45, wspace = 0.55)
        for m_id in range(0,n_rows):
                axes[m_id].plot(N_is,Scores_all[:,m_id], linestyle='--', marker='o', color='lightseagreen', label=Score_label)
                axes[m_id].xaxis.set_major_locator(MaxNLocator(integer=True))
                if m_id==n_rows-1:
                    axes[m_id].set_xlabel("Number of intermediate states")
                axes[m_id].set_ylabel(model_str[m_id]) 
                if m_id==0:
                    axes[m_id].legend()
        f.text(0, 0.5, Score_label + ' values', va='center', rotation='vertical')
        f.patch.set_facecolor('white')
        if opt_save:
            fig = plt.gcf()
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Nis_per_model.svg', bbox_inches="tight")
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Nis_per_model.pdf', bbox_inches="tight")
    else:
        #fig,ax = plt.figure()
        for m_id in range(0,len(model_str)):
            plt.plot(N_is,Scores_all[:,m_id], linestyle='--', marker='o',color=colors[m_id], label=model_str[m_id])
            plt.xlabel("Number of intermediate states")
            plt.ylabel(Score_label + ' value') 
            plt.legend(bbox_to_anchor=(1.3, 1))
            #ax = plt.figure().gca()
            #ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        if opt_save:
            fig = plt.gcf()
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Nis_all_models.svg', bbox_inches="tight")
            fig.savefig(path_matlab_result + '/' + Score_label + '_Scores_vs_Nis_all_models.pdf', bbox_inches="tight")
    return;

In [ ]:
def calculate_Score_category_percentages(S):
    IDX_min = list()
    IDX_plausible = list()
    N_individuals = np.shape(S)[2]
    N_models = np.shape(S)[1]
    best = list()
    plausible = list()
    rejected = list()
    for i_id in range(0,N_individuals):
        IDX_min = IDX_min + np.where(S == np.min(S[0,0:N_models,i_id]))[1].tolist()
        IDX_plausible = IDX_plausible + np.where((S[0,0:N_models,i_id]-np.min(S[0,0:N_models,i_id])<10) & (S[0,0:N_models,i_id]!=np.min(S[0,0:N_models,i_id])))[0].tolist()
    for m_id in range(0,N_models):
        best.append(100*(IDX_min.count(m_id)/N_individuals))
        plausible.append(100*(IDX_plausible.count(m_id)/N_individuals))
        rejected.append(100*((N_individuals-IDX_min.count(m_id)-IDX_plausible.count(m_id))/N_individuals))
    return best, plausible, rejected

In [ ]:
def plot_change_in_Score(Score,Score_str,I_names,M_names,N_is,results_str,opt_perModel,opt_subplots,colors,opt_save):
    if opt_perModel:
        if opt_subplots:
            n_rows = 2
            n_cols = int(np.ceil(len(M_names)/2))
            f, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*2+4, n_rows*2+4), sharex=True, sharey=True)
            f.patch.set_facecolor('white')
            f.subplots_adjust(hspace = .25, wspace = 0.15)
        else:
            f,ax = plt.subplots()
        S_change = np.zeros((len(M_names),len(N_is)-1,len(I_names)),dtype=float)
        S_change_mean = np.zeros((len(M_names),len(N_is)-1),dtype=float)
        S_change_max = np.zeros((len(M_names),len(N_is)-1),dtype=float)
        S_change_min = np.zeros((len(M_names),len(N_is)-1),dtype=float)
        for m_id in range(0,len(M_names)):
            S = np.matrix(Score[:,m_id,:])
            for i in range(0,len(N_is)-1):
                if Score_str=='LOGL':
                    S_change[m_id,i,:] = (S[i+1,:]-S[i,:])/np.abs(S[i])*100/(N_is[i+1]-N_is[i])
                    S_change_mean[m_id,i] = np.mean((S[i+1,:]-S[i,:])/np.abs(S[i]))*100/(N_is[i+1]-N_is[i])
                    S_change_max[m_id,i] = np.max((S[i+1,:]-S[i,:])/np.abs(S[i]))*100/(N_is[i+1]-N_is[i])
                    S_change_min[m_id,i] = np.min((S[i+1,:]-S[i,:])/np.abs(S[i]))*100/(N_is[i+1]-N_is[i])
                else:
                    S_change[m_id,i,:] = (S[i,:]-S[i+1,:])/np.abs(S[i])*100/(N_is[i+1]-N_is[i])
                    S_change_mean[m_id,i] = np.mean((S[i,:]-S[i+1,:])/np.abs(S[i]))*100/(N_is[i+1]-N_is[i])
                    S_change_max[m_id,i] = np.max((S[i,:]-S[i+1,:])/np.abs(S[i]))*100/(N_is[i+1]-N_is[i])
                    S_change_min[m_id,i] = np.min((S[i,:]-S[i+1,:])/np.abs(S[i]))*100/(N_is[i+1]-N_is[i])
            #print('Model '+M_names[m_id])
            #print('average increase/ decrease in logL value (in %) per n_is increase: ')
            #print('mean: '+ str(LogL_change_mean[m_id,]))
            #print('max: '+ str(LogL_change_max[m_id,]))
            #print('min: '+ str(LogL_change_min[m_id,]))
            if opt_subplots:
                if m_id>=np.round(len(M_names)/2):
                    r_id=1
                    c_id = m_id-n_cols
                else:
                    r_id=0
                    c_id = m_id

                axes[r_id,c_id].plot(N_is[0:-1],S_change_mean[m_id,],'-',linewidth=3,color=colors[m_id])
                for i_id in range(0,np.size(S_change,2)):
                    axes[r_id,c_id].plot(N_is[0:-1],S_change[m_id,:,i_id],'-o',linewidth=1,ms=3,color=colors[m_id])

                #axes[r_id,c_id].plot(N_is[0:-1],S_change_mean[m_id,],'o--',color=colors[m_id])
                #axes[r_id,c_id].plot(N_is[0:-1],S_change_max[m_id,],'-',color=colors[m_id],alpha=0.5)
                #axes[r_id,c_id].plot(N_is[0:-1],S_change_min[m_id,],'-',color=colors[m_id],alpha=0.5)
                axes[r_id,c_id].fill_between(N_is[0:-1],S_change_min[m_id,],S_change_max[m_id,],color=colors[m_id],alpha=0.1)
                #axes[r_id,c_id].plot(N_is[0:-1],np.zeros(np.shape(N_is[0:-1])),'-k')
                if r_id==1 and c_id==2:
                    axes[r_id,c_id].set_xlabel('Number of intermediate states');
                if c_id==0 and r_id==0:
                    if Score_str=='LOGL':
                        axes[r_id,c_id].set_ylabel('Percentage increase in '+Score_str+' / n_is increase');
                    else:
                        axes[r_id,c_id].set_ylabel('Percentage decrease in '+Score_str+' / n_is increase');
                ymin,ymax = axes[r_id,c_id].get_ylim()
                axes[r_id,c_id].set_ylim([0, ymax])
                axes[r_id,c_id].set_title('Model '+M_names[m_id])
                axes[r_id,c_id].xaxis.set_major_locator(MaxNLocator(integer=True))
            else:
                ax.plot(N_is[0:-1],S_change_mean[m_id,],'--o',linewidth=2,color=colors[m_id])
                if m_id==len(M_names)-1:
                    if Score_str=='LOGL':
                        ax.set_ylabel('Percentage increase in '+Score_str+' / n_is increase');
                    else:
                        ax.set_ylabel('Percentage decrease in '+Score_str+' / n_is increase');
                    ymin,ymax = ax.get_ylim()
                    ax.set_ylim([0, ymax])
                    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        if opt_subplots:
            if (n_rows-1)*n_cols+c_id<len(M_names)-1:
                #print((n_rows-1)*n_cols+c_id)
                f.delaxes(axes[r_id,c_id+1])
    else:
        S_change_mean = np.zeros((len(N_is)-1,1),dtype=float)
        S_change_max = np.zeros((len(N_is)-1,1),dtype=float)
        S_change_min = np.zeros((len(N_is)-1,1),dtype=float)
        S = np.matrix(np.sum(Score[:,:,:], axis=1))
        for i in range(0,len(N_is)-1):
            if Score_str=='LOGL':
                S_change_mean[i] = np.mean((S[i+1]-S[i])/np.abs(S[i]),axis=1)*100/(N_is[i+1]-N_is[i])
                S_change_max[i] = np.max((S[i+1]-S[i])/np.abs(S[i]),axis=1)*100/(N_is[i+1]-N_is[i])
                S_change_min[i] = np.min((S[i+1]-S[i])/np.abs(S[i]),axis=1)*100/(N_is[i+1]-N_is[i])
            else: 
                S_change_mean[i] = np.mean((S[i]-S[i+1])/np.abs(S[i]),axis=1)*100/(N_is[i+1]-N_is[i])
                S_change_max[i] = np.max((S[i]-S[i+1])/np.abs(S[i]),axis=1)*100/(N_is[i+1]-N_is[i])
                S_change_min[i] = np.min((S[i]-S[i+1])/np.abs(S[i]),axis=1)*100/(N_is[i+1]-N_is[i])
        ax = plt.figure().gca()
        ax.plot(N_is[0:-1],S_change_mean,'o--',color='black')
        #ax.plot(N_is[0:-1],S_change_max,'-',color='steelblue',alpha=0.5)
        #ax.plot(N_is[0:-1],S_change_min,'-',color='steelblue',alpha=0.5)
        ax.fill_between(N_is[0:-1],S_change_min.flatten(),S_change_max.flatten(),color='black',alpha=0.1)
        #ax.plot(N_is[0:-1],np.zeros(np.shape(N_is[0:-1])),'-k')
        
        ax.set_xlabel('Number of intermediate states');
        if Score_str=='LOGL':
            ax.set_ylabel('Mean increase in '+Score_str+' / n_is increase');
        else:
            ax.set_ylabel('Mean decrease in '+Score_str+' / n_is increase');
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    #save figure    
    if opt_save:
        fig = plt.gcf()
        fig.savefig(results_str + '/' + 'ChangeIn'+Score_str+'With_Nis_increase.svg', bbox_inches="tight")
        fig.savefig(results_str + '/' + 'ChangeIn'+Score_str+'With_Nis_increase.pdf', bbox_inches="tight")

In [10]:
def plot_scores_vs_individuals_per_model(path_matlab,F_str,SSF_str_matlab,Scores,Score_label,colors,n_is,N_is,bool_subplots,opt_save):
    idx_nIS_ofInterest = [i for i, n in enumerate(N_is) if n == n_is] 
    if bool_subplots:
        n_rows = 1
        n_cols = len(SSF_str_matlab)#individuals
        f, axes = plt.subplots(n_rows, n_cols, figsize=(40, 7))
        f.subplots_adjust(hspace = .45, wspace = 0.35)
        for i_id in range(0,n_cols):
            for m_id in range(0,len(model_str)):
                axes[i_id].plot(i_id,Scores[idx_nIS_ofInterest[0],m_id,i_id], marker='o',markersize=12,color=colors[m_id])
                if m_id==n_rows-1:
                    axes[i_id].set_xlabel(SSF_str_matlab[i_id])
                if m_id==0:
                    axes[0].set_ylabel(Score_label + ' values')  
                if m_id==len(model_str)-1:
                    axes[n_cols-1].legend(bbox_to_anchor=(1.6, 1),labels=model_str)
        f.text(0.5, 0.95, str(N_is[idx_nIS_ofInterest[0]]) +' intermediate states', va='center')
        f.patch.set_facecolor('white')
        if opt_save:
            fig = plt.gcf()
            fig.savefig(path_matlab + '/' + Score_label + '_Scores_vs_Individuals_per_model.svg', bbox_inches="tight")
            fig.savefig(path_matlab + '/' + Score_label + '_Scores_vs_Individuals_per_model.pdf', bbox_inches="tight")
    else:
        for m_id in range(0,len(model_str)):
            x=range(0,len(SSF_str_matlab))
            plt.plot(x,Scores[idx_nIS_ofInterest[0],m_id,:].flatten(), marker='o', linestyle = '', color=colors[m_id])
            if m_id==0:
                plt.ylabel(Score_label+' values')  
            if m_id==len(model_str)-1:
                plt.xlabel('Individuals' )
                plt.xticks(x,SSF_str_matlab,rotation=90)
                plt.legend(bbox_to_anchor=(1.6, 1),labels=model_str,numpoints=1)
        plt.title(str(N_is[idx_nIS_ofInterest[0]]) +' intermediate states')
        if opt_save:
            fig = plt.gcf()
            fig.savefig(path_matlab + '/' + Score_label + '_Scores_vs_Individuals_all_models.svg', bbox_inches="tight")
            fig.savefig(path_matlab + '/' + Score_label + '_Scores_vs_Individuals_all_models.pdf', bbox_inches="tight")
    return;

In [ ]:
def plot_time_vs_Nis(Time_all,Time,M_names,N_is,N_indi,results_str,colors,opt_subplots,opt_save):

    if opt_subplots:
        n_rows = 2
        n_cols = int(np.ceil(len(M_names)/2))
        f, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*2+5, n_rows*2+4))
        f.patch.set_facecolor('white')
        f.subplots_adjust(hspace = .35, wspace = 0.55)
    else:
        f,ax = plt.subplots()
    for m_id in range(0,len(M_names)):
        #q_05 = np.zeros((len(N_is),len(M_names)),float)
        #q_95 = np.zeros((len(N_is),len(M_names)),float)
        #for m_id in range(0,len(M_names)):
        #    q_05[:,m_id] = np.quantile(Time[:,m_id,:],0.05,axis=1)
        #    q_95[:,m_id] = np.quantile(Time[:,m_id,:],0.95,axis=1)
        if opt_subplots:
            if m_id>=np.round(len(M_names)/2):
                r_id=1
                c_id = m_id-n_cols
            else:
                r_id=0
                c_id = m_id
        T_min = np.min(Time[:,m_id,:],axis=1)
        T_max = np.max(Time[:,m_id,:],axis=1)
        if opt_subplots:
            axes[r_id,c_id].plot(N_is,Time_all[:,m_id]/N_indi,'-',linewidth=3,color=colors[m_id])
            for i_id in range(0,np.size(Time,2)):
                axes[r_id,c_id].plot(N_is,Time[:,m_id,i_id],'-o',linewidth=1,ms=3,color=colors[m_id])

            #axes[r_id,c_id].fill_between(N_is, q_05[:,m_id], q_95[:,m_id],color=colors[m_id],alpha=0.1)
            axes[r_id,c_id].fill_between(N_is, T_min, T_max,color=colors[m_id],alpha=0.1)
            if r_id==1 and c_id==2:
                axes[r_id,c_id].set_xlabel("Number intermediate states")
            if c_id==0:# and r_id==0:
                axes[r_id,c_id].set_ylabel("Time per sample [h]")
            axes[r_id,c_id].set_title('Model '+M_names[m_id])
            axes[r_id,c_id].xaxis.set_major_locator(MaxNLocator(integer=True))
        else:
            ax.plot(N_is,Time_all[:,m_id]/N_indi,'--o',linewidth=2,color=colors[m_id])
            if m_id==len(M_names)-1:
                ax.set_xlabel("Number intermediate states")
                ax.set_ylabel("Time per sample [h]")
                ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    if opt_subplots:
        if ((n_rows-1)*n_cols+c_id<len(M_names)-1):
            f.delaxes(axes[r_id,c_id+1])
        
    #save figure    
    if opt_save:
        fig = plt.gcf()
        fig.savefig(results_str + '/' + 'Time_vs_Nis.svg', bbox_inches="tight")
        fig.savefig(results_str + '/' + 'Time_vs_Nis.pdf', bbox_inches="tight")

In [ ]:
def plot_percentage_identifiable_parameters_per_intermediate_state(identifiable_percent,M_names,colors,opt_subplots,opt_save):
    if opt_subplots:
        n_rows = 2
        n_cols = int(np.ceil(len(M_names)/2))
        f, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*2+4, n_rows*2+4), sharex=True, sharey=True)
        f.patch.set_facecolor('white')
        f.subplots_adjust(hspace = .25, wspace = 0.15)
    else:
        f, ax = plt.subplots()
    ID_PAR_mean = np.zeros((len(M_names),len(N_is)),dtype=float)
    ID_PAR_max = np.zeros((len(M_names),len(N_is)),dtype=float)
    ID_PAR_min = np.zeros((len(M_names),len(N_is)),dtype=float)
    for m_id in range(0,len(M_names)):
        for i in range(0,len(N_is)):
            ID_PAR_mean[m_id,i] = np.mean(identifiable_percent[m_id,i,:])
            ID_PAR_max[m_id,i] = np.max(identifiable_percent[m_id,i,:])
            ID_PAR_min[m_id,i] = np.min(identifiable_percent[m_id,i,:])
        if opt_subplots:
            if m_id>=np.round(len(M_names)/2):
                r_id=1
                c_id = m_id-n_cols
            else:
                r_id=0
                c_id = m_id
            axes[r_id,c_id].plot(N_is,ID_PAR_mean[m_id,],'-',linewidth=3,color=colors[m_id])
            for i_id in range(0,np.size(identifiable_percent,2)):
                axes[r_id,c_id].plot(N_is,identifiable_percent[m_id,:,i_id],'-o',linewidth=1,ms=3,color=colors[m_id])
            #axes[r_id,c_id].plot(N_is,ID_PAR_max[m_id,],'-',alpha=0.1,color=colors[m_id])
            #axes[r_id,c_id].plot(N_is,ID_PAR_min[m_id,],'-',alpha=0.1,color=colors[m_id])
            axes[r_id,c_id].fill_between(N_is,ID_PAR_min[m_id,],ID_PAR_max[m_id,],color=colors[m_id],alpha=0.1)
            if r_id==1 and c_id==2:
                axes[r_id,c_id].set_xlabel('Number of intermediate states');
            if c_id==0 and r_id==0:
                axes[r_id,c_id].set_ylabel('Identifiable parameters [%]',Fontsize=15)
            axes[r_id,c_id].set_title('Model '+M_names[m_id])
            axes[r_id,c_id].xaxis.set_major_locator(MaxNLocator(integer=True))
        else:
            ax.plot(N_is,ID_PAR_mean[m_id,],'o--',linewidth=2,color=colors[m_id])
            if m_id==len(M_names)-1:
                ax.set_xlabel('Number of intermediate states');
                ax.set_ylabel('Identifiable parameters [%]',Fontsize=15)
                ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    if opt_subplots:
        if (n_rows-1)*n_cols+c_id<len(M_names)-1:
            #print((n_rows-1)*n_cols+c_id)
            f.delaxes(axes[r_id,c_id+1])
    #save figure    
    if opt_save:
        fig = plt.gcf()
        fig.savefig(results_str + '/' + 'PercentageIdentifiableParameters_vs_Nis.svg', bbox_inches="tight")
        fig.savefig(results_str + '/' + 'PercentageIdentifiableParameters_vs_Nis.pdf', bbox_inches="tight")            

In [11]:
def highlight_small_diffs_to_min(s):
    '''
    highlight other plausible models (difference in BIC to best model <10) in a Series yellow.
    '''
    diff_to_min_below_10 = (s-np.min(s)) < 10
    return ['background-color: orange' if v else '' for v in diff_to_min_below_10 ]

In [12]:
def highlight_min2(s):
    '''
    highlight other plausible models (difference in BIC to best model <10) in a Series yellow.
    '''
    min_bool = (s==s.min())
    return ['background-color: red' if v else '' for v in min_bool ]

In [13]:
def highlight_max2(s):
    '''
    highlight other plausible models (difference in BIC to best model <10) in a Series yellow.
    '''
    min_bool = (s==s.max())
    return ['background-color: red' if v else '' for v in min_bool ]

In [14]:
def get_styled_Score_table(option,N_is,n_is,Scores,Scores_all,Score_label,SSF_str,M_names_opt,bool_fit_repetitions_seperately):
    colNames_list = list()
    if option == 'compare Nis':
        a=1
        df = pd.DataFrame({'model': ['model '+ m_names for m_names in M_names_opt]})
        for ID_IS in N_is:
            idx_nIS_ofInterest = [i for i, n in enumerate(N_is) if n == ID_IS] 
            df[Score_label + '_' + str(ID_IS) + '_iS'] = Scores_all[idx_nIS_ofInterest[0],:]
            colNames_list.append(Score_label + '_' + str(ID_IS) + '_iS')
    elif option == 'compare individuals':
        df = pd.DataFrame({'model': ['model '+ m_names for m_names in M_names_opt]})
        idx_nIS_ofInterest = [i for i, n in enumerate(N_is) if n == n_is] 
        a=0
        for ID_I in range(0,np.shape(Scores)[2]):#len(SSF_str)):
            df[Score_label + '_' + SSF_str[ID_I]] = Scores[idx_nIS_ofInterest[0],:,ID_I]
        if bool_fit_repetitions_seperately:
            df.rename(columns=lambda x: x[len(x)-5:len(x)-2] if x != 'model' else x, inplace=True)
        else:
            df.rename(columns=lambda x: x[len(x)-3:len(x)] if x != 'model' else x, inplace=True)
        colNames_list = df.columns#.str.encode('ascii','ignore')
    elif option == 'compare sim_models to opt_models':
        df = pd.DataFrame({'opt. with\ sim. from': ['model '+ m_names for m_names in M_names_opt]})
        a=0
        #for ID_I in range(0,np.shape(Scores)[2]):
            #print(Score_label + '_' + SSF_str[ID_I])
        for ID_simModel in range(0,np.shape(Scores)[0]):
            df['model '+M_names_opt[ID_simModel]] = np.mean(Scores,axis=2)[ID_simModel,:]
        colNames_list = df.columns
    #pd.options.display.float_format = "{:,.2f}".format
    df = df.round(decimals=0)
    if Score_label=='LOGL':
        df_styled2 = (df.style
                    .apply(highlight_max2, subset=colNames_list[1:len(colNames_list)], axis=a)   
                    .set_precision(precision=0)
                    .set_properties(**{'text-align': 'left'}))
    else:
        df_styled2 = (df.style
                    .apply(highlight_small_diffs_to_min, subset=colNames_list[1:len(colNames_list)], axis=a)
                    .apply(highlight_min2, subset=colNames_list[1:len(colNames_list)], axis=a)
                    .set_precision(precision=0)
                    .set_properties(**{'text-align': 'left'}))
    return df_styled2

In [16]:
def plot_stackedBarplot_of_Score_category_percentages(best, plausible, rejected, criterion, main_folder_matlab, F_str, str_header,opt_save):
    ind = [M_name for M_name in M_names]

    plt.bar(ind, np.array(rejected), width=0.6, label='rejected', color='grey', bottom=np.array(best)+np.array(plausible))
    plt.bar(ind, np.array(plausible), width=0.6, label='plausible', color='orange', bottom=np.array(best))
    plt.bar(ind, np.array(best), width=0.6, label='best', color='red')

    plt.xticks(ind, M_names)
    plt.ylabel("Percentage")
    plt.xlabel("Models")
    plt.legend(loc="upper right")
    plt.title(criterion + " result for all models based on"+ str_header +"individuals")
    if opt_save:
        fig = plt.gcf()
        fig.savefig(main_folder_matlab + F_str + '/' + criterion + str_header + '_score_category_percantages.svg', bbox_inches="tight")
        fig.savefig(main_folder_matlab + F_str + '/' + criterion + str_header + '_score_category_percantages.pdf', bbox_inches="tight")

    plt.show()

In [17]:
def hex_to_rgb(hex):
    hex = hex.lstrip('#')
    h_length = len(hex)
    rgb_val = tuple(int(hex[i:i+int(h_length/3)], 16)/255 for i in range(0, h_length, int(h_length/3)))
    return rgb_val

In [ ]:
def getWeightDataframe(W_raw,W_data):
    W = W_raw + (np.tile(W_data,(np.shape(W_raw)[1],1))).T #uncomment if number of used data points should be used as weight
    W = W/np.sum(W)
    return W;

In [ ]:
def loadPltSettings(fontSize,markerSize):
    plt.style.use('classic')
    plt.rcParams['grid.alpha'] = 0
    plt.rcParams['font.size'] = fontSize
    plt.rcParams['axes.edgecolor'] ='black'
    plt.rcParams['axes.facecolor'] ='white'
    plt.rcParams['axes.labelcolor'] = 'black'
    plt.rcParams['figure.edgecolor'] = 'white'
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['figure.titlesize'] = fontSize
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['axes.labelsize']= fontSize
    plt.rcParams['legend.fontsize'] = fontSize
    plt.rcParams['boxplot.flierprops.markersize']= markerSize
    return plt